In [4]:
import pysd
import numpy as np
import scipy.optimize
import math
import pandas as pd
import random
random.seed(100)
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

# model loading

In [5]:
#model = pysd.read_vensim('vensim_models/mngChain/mngChain_wolookup2.mdl')
model_gen = pysd.read_vensim('vensim_models/optimization/InventoryManagementWeek5-Generator.mdl')
model_est = pysd.read_vensim('vensim_models/optimization/InventoryManagementWeek5-Estimator.mdl')
data_gen = model_gen.run()
data_est = model_est.run()

In [6]:
data_gen

,FINAL TIME,INITIAL TIME,SAVEPER,TIME STEP,Noise Standard Deviation,White Noise,Production Rate,Customer Order Rate,Change in Process Noise,Noise Correlation Time,...,Shipment Rate[1.0],Shipment Rate[1.2],Shipment Rate[1.4],Shipment Rate[1.6],Shipment Rate[1.8],Shipment Rate[2.0],Target Delivery Delay,Time to Average Order Rate,WIP Adjustment Time,Work in Process Inventory
0.0000,100,0,0.0625,0.0625,1,-14.630996,10000.0,10000,1.463100,10,...,8500.000000,9300.000000,9700.000000,9900.000000,10000.0,10000.0,2,8,2,80000.0
0.0625,100,0,0.0625,0.0625,1,-14.630996,10914.437257,10000,1.472244,10,...,8539.843750,9320.343750,9709.093750,9903.093750,10000.0,10000.0,2,8,2,80000.0
0.1250,100,0,0.0625,0.0625,1,-14.630996,[<xarray.DataArray ()>\narray(11822.523438)\nC...,10000,1.481446,10,...,8578.629150,9340.096260,9717.911846,9906.091787,10000.0,10000.0,2,8,2,[<xarray.DataArray ()>\narray(79918.43360891)\...
0.1875,100,0,0.0625,0.0625,1,-14.630996,[<xarray.DataArray ()>\narray(12721.57595274)\...,10000,1.490705,10,...,8616.384314,9359.274712,9726.462643,9908.997072,10000.0,10000.0,2,8,2,[<xarray.DataArray ()>\narray(79756.01420592)\...
0.2500,100,0,0.0625,0.0625,1,-14.630996,[<xarray.DataArray ()>\narray(13608.93780109)\...,10000,1.500021,10,...,8653.136605,9377.895791,9734.754244,9911.812476,10000.0,10000.0,2,8,2,[<xarray.DataArray ()>\narray(79513.61604449)\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.7500,100,0,0.0625,0.0625,1,-14.630996,[<xarray.DataArray ()>\narray(nan)\nCoordinate...,10000,30470.509985,10,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.0,2,8,2,[<xarray.DataArray ()>\narray(nan)\nCoordinate...
99.8125,100,0,0.0625,0.0625,1,-14.630996,[<xarray.DataArray ()>\narray(nan)\nCoordinate...,10000,30660.950673,10,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.0,2,8,2,[<xarray.DataArray ()>\narray(nan)\nCoordinate...
99.8750,100,0,0.0625,0.0625,1,-14.630996,[<xarray.DataArray ()>\narray(nan)\nCoordinate...,10000,30852.581615,10,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.0,2,8,2,[<xarray.DataArray ()>\narray(nan)\nCoordinate...
99.9375,100,0,0.0625,0.0625,1,-14.630996,[<xarray.DataArray ()>\narray(nan)\nCoordinate...,10000,31045.410250,10,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.0,2,8,2,[<xarray.DataArray ()>\narray(nan)\nCoordinate...


In [7]:
data_est

,FINAL TIME,INITIAL TIME,SAVEPER,TIME STEP,Cumulative Payoff,Customer Order Rate,Min Err,Calib Payoff[0.0],Calib Payoff[0.2],Calib Payoff[0.4],...,Shipment Rate[1.2],Shipment Rate[1.4],Shipment Rate[1.6],Shipment Rate[1.8],Shipment Rate[2.0],Target Delivery Delay,Time to Average Order Rate,White Noise,WIP Adjustment Time,Work in Process Inventory
0.0000,100,0,0.0625,0.0625,0,10000,100,2.634324e+00,4.803557,4.803557,...,9300.000000,9700.000000,9900.000000,10000.0,10000.0,2,8,12.421211,2,80000.0
0.0625,100,0,0.0625,0.0625,[<xarray.DataArray ()>\narray(0.16464524)\nCoo...,10000,100,1.401841e+00,1.545418,1.545418,...,9320.343750,9709.093750,9903.093750,10000.0,10000.0,2,8,12.421211,2,80000.0
0.1250,100,0,0.0625,0.0625,[<xarray.DataArray ()>\narray(0.25226031)\nCoo...,10000,100,2.042561e+00,2.868525,2.868525,...,9340.096260,9717.911846,9906.091787,10000.0,10000.0,2,8,12.421211,2,[<xarray.DataArray ()>\narray(80024.10629318)\...
0.1875,100,0,0.0625,0.0625,[<xarray.DataArray ()>\narray(0.37992039)\nCoo...,10000,100,5.221516e-02,0.165344,0.165344,...,9359.274712,9726.462643,9908.997072,10000.0,10000.0,2,8,12.421211,2,[<xarray.DataArray ()>\narray(80073.60513861)\...
0.2500,100,0,0.0625,0.0625,[<xarray.DataArray ()>\narray(0.38318384)\nCoo...,10000,100,1.160966e+00,1.395247,1.395247,...,9377.895791,9734.754244,9911.812476,10000.0,10000.0,2,8,12.421211,2,[<xarray.DataArray ()>\narray(80149.78684785)\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.7500,100,0,0.0625,0.0625,[<xarray.DataArray ()>\narray(4.82217478)\nCoo...,10000,100,1.201209e-29,1.222953,1.222953,...,10000.000000,10000.000000,10000.000000,10000.0,10000.0,2,8,12.421211,2,[<xarray.DataArray ()>\narray(94666.69600648)\...
99.8125,100,0,0.0625,0.0625,[<xarray.DataArray ()>\narray(4.82217478)\nCoo...,10000,100,2.912479e-31,0.645260,0.645260,...,10000.000000,10000.000000,10000.000000,10000.0,10000.0,2,8,12.421211,2,[<xarray.DataArray ()>\narray(94666.69600648)\...
99.8750,100,0,0.0625,0.0625,[<xarray.DataArray ()>\narray(4.82217478)\nCoo...,10000,100,6.987256e-28,0.159084,0.159084,...,10000.000000,10000.000000,10000.000000,10000.0,10000.0,2,8,12.421211,2,[<xarray.DataArray ()>\narray(94666.69600648)\...
99.9375,100,0,0.0625,0.0625,[<xarray.DataArray ()>\narray(4.82217478)\nCoo...,10000,100,8.022176e-27,1.863298,1.863298,...,10000.000000,10000.000000,10000.000000,10000.0,10000.0,2,8,12.421211,2,[<xarray.DataArray ()>\narray(94666.69600648)\...


# Driving Data import

In [5]:
customer_order_rate = pd.read_csv('data/example_retail_sales.csv')